In [ ]:
## Import and install necessary modules

install.packages("tidytext")
install.packages("tm")
install.packages("SnowballC")
install.packages("wordcloud")
install.packages("RColorBrewer")
install.packages("topicmodels")
install.packages("ldatuning")
install.packages("SentimentAnalysis")

library(tidytext)
library(tm)
library(SnowballC)
library(wordcloud)
library(RColorBrewer)
library(topicmodels)
library(ldatuning)
library(SentimentAnalysis)

We begin by installing the necessary modules for our analysis.

In [ ]:
## Read Data

reviews <- read.csv("yelp.csv", stringsAsFactors = F)         #Read calls from subset of first dataset                                                                          in which all calls entirely composed of                                                                          stopwords have been removed
reviews <- as.matrix(calls[5])                                #Convert data to matrix
reviews.vec <- VectorSource(calls)                            #Create data vector
reviews.corpus <- Corpus(calls.vec)                           #Create corpus

The next step is to read in our data and convert it into a corpus so that we can apply text mining methods to it. 

In [ ]:
## Clean Data

reviews.corpus <- tm_map(reviews.corpus, tolower)                               #Remove capital letters
reviews.corpus <- tm_map(reviews.corpus, removePunctuation)                     #Remove punctuation
reviews.corpus <- tm_map(reviews.corpus, removeNumbers)                         #Remove numbers
reviews.corpus <- tm_map(reviews.corpus, removeWords, stopwords("english"))     #Remove stopwords
reviews.corpus <- tm_map(reviews.corpus, stemDocument)                          #Stem words
reviews.corpus <- tm_map(reviews.corpus, stripWhitespace)                       #Remove whitespace

Once the data has been read, it is necessary to clean it. Preprocessing makes the text more uniform and ultimately improves results. Here, we convert all text to lower case, remove punctuation, numbers, and stopwords, stem words, and remove extra whitespace. The benefits to each process are:

**Lower case:** Converting all text to lower case ensures that words with different capitalizations are not counted differently. For example, we do not want "Text" and "text" to be counted as different words as this can significantly alter results. Perhaps this process is not desired if one wishes to understand what people start their sentences with, but that is not the aim of our project.

**Remove Punctuation:** Again, this further renders our data uniform. It is also possible that failing to remove punctuation could count "text-mining," "text mining", and "text mining." differently.

**Remove Numbers:** Although extracting numbers in a review may uncover certain information, they are difficult to work with and would most likely have very little impact on our project.

**Remove Stop Words:** Stop words are words with very little predictive value. These often clutter the dataset without providing any insight. An added benefit is that working with the dataset after removing these words is more computationally efficient.

**Stem Words:** Words of various conjugations and forms have the same meaning for our purposes so we do not want them to be counted separately. For example, "visit," "visited," and "visitor" all convey the same general meaning and thus we would like them grouped together. This method may backfire for certain words, but it generally improves results overall.

**Stripping White Space:** After performing these pre-processing operations, we are left with many empty spaces which make the data difficult to read. Stripping unnecessary whitespace remedies this issue and makes our data more coherent.

In [ ]:
## Word Frequencies

#Create Term Document Matrix
tdm <- TermDocumentMatrix(calls.corpus, 
        control=list(weighting=weightTfIdf))            #Specify weighting (currently TF-IDF)

removeSparseTerms(tdm, 0.99)                           #Remove infrequent terms

tdm <- as.matrix(tdm)                                  #Convert term document matrix to matrix form


v <- sort(rowSums(tdm),decreasing=TRUE)                #Sort in order
d <- data.frame(word = names(v),freq=v)                #Convert to dataframe
head(d, 20)                                            #Preview data
inspect(TermDocumentMatrix(calls.corpus,               #Inspect term document matrix
        control = list(weighting = weightTfIdf)))

Now that our dataset is ready for analysis, we create a term-document matrix which shows the TF-IDF weight of each word in each review. Here, we remove infrequent terms to keep our matrix small while preserving its statistical abilities. Before moving on, we also sort our data, convert it into a dataframe, and inspect our term-document matrix.

In [ ]:
## Create Word Cloud

set.seed(9162)
wordcloud(words=d$word, freq=d$freq, min.freq=1,max.words=35, 
          random.order=FALSE, rot.per=0.3, colors=brewer.pal(8, "Dark2"))

Word clouds are an effective exploratory methods in text-mining as they visually show important words in the corpus. As a result, we receive a quick, general idea of the kinds of words our dataset contains. More important (e.g. prevalent, high TF-IDF weight) words are larger and closer to the center of the cloud.

In [ ]:
## Topic Modeling Preparation

dtm <- DocumentTermMatrix(review.corpus, control=list(weighting = weightTf)) #(Different than preivous tdm)
rowTotals <- apply(dtm, 1, sum)
dtm.new   <- dtm[rowTotals > 0, ]     #Modify new term document matrix for LDA

#Finding ideal number of topics (currently evaluating between 5-15)
#Takes a long time to run

idealk <- FindTopicsNumber(dtm.new, topics = seq(from=5, to=15, by=1),
  metrics = c("CaoJuan2009", "Arun2010", "Deveaud2014"),
  method = "VEM", control = list(seed=77), verbose = TRUE)

FindTopicsNumber_plot(idealk)

Although we performed our topic modeling in Python, we wanted to see if we could find an optimal number of topics, k. This function uses various metrics to find the answer; however, we did not use these results to proceed with our analysis for various reasons. 

**1:** This method indicates our number of topics should be much higher than 15 (the maximum we tested), but we opted to use 10 because using many topics was computationally expensive

**2:** This function is designed for finding the best number of LDA topics. We used both LDA and NMF in our analysis.

**3:** This method takes a long time to run. We are currently running it between 5 and 15 topics and the marginal increase in running time increases with each additional topic

In [ ]:
## Sentiment Analysis

sent <- analyzeSentiment(calls.corpus)     #Sentiment Analysis
plotSentiment(sent)                        #Plot

binSent <- convertToBinaryResponse(sent)   #Binary response
dirSent <- convertToDirection(sent)        #Directional response

#Create sentiment datasets
sentByMethod <- cbind(binSent[,1], binSent[,2], binSent[,5], binSent[8], binSent[,12])
sentMethods2 <- cbind(dirSent[,1], dirSent[,2], dirSent[,5], dirSent[8], dirSent[,12])

#Write sentiment datasets to CSV files
write.csv(sentByMethod, file="Binary Review Sentiments.csv")
write.csv(sentMethods2, file="Directional Review Sentiments.csv")

Finally, we used sentiment analysis to understand how Yelp reviewers felt in their experiences. A quick look at the dataset shows that most reviews were positive, but we wanted a more in-depth understanding as well.

The first thing we did was plot the sentiment of each review. This was not extremely useful for our purposes but could have been for identifying trends over time if our reviews came with a time-stamp.

We then studied the binary and directional sentiments of each review through four different sentiment analysis dictionary. Each dictionary defines different words with a certain positivity or negativity scores, which the model aggregates. Using a binary response gives a score of "positive" or "negative" for each review and a directional response gives a score of "positive," "neutral," or "negative."

We then output the results of our analysis in CSV files. Each file contains the review number, the number of words in the review which were referenced by all dictionaries, and the label each dictionary provided the review. As a result we are left with four scores for each review. In Excel, we assigned values as such: positive = 1, neutral = 0, negative = -1 and averaged these scores to arrive at an ultimate "average sentiment."

Again, we saw that reviews are mostly positive, but this look provides a more concrete conclusion that simply using star ratings.